In [ ]:
####################################################################################################

#    Copyright 2019 Srijan Verma and EMBL-European Bioinformatics Institute

#    Licensed under the Apache License, Version 2.0 (the "License");
#    you may not use this file except in compliance with the License.
#    You may obtain a copy of the License at

#        http://www.apache.org/licenses/LICENSE-2.0

#    Unless required by applicable law or agreed to in writing, software
#    distributed under the License is distributed on an "AS IS" BASIS,
#    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#    See the License for the specific language governing permissions and
#    limitations under the License.

####################################################################################################

In [31]:
import pandas as pd
df_overlap = pd.read_csv('/feature_selection/ORF/Ens-Ref/ens_ref_ORF&overlap_cleaned(ens).csv', index_col=0)

In [32]:
df_overlap.head(2)

,ENS lincRNA gene ID,Ref lncRNA gene ID,Chr region,No. of transcripts in lincrna,No. of transcripts in lncrna,ENS-Ref Overlap ?,same_opp_overlap,Overlap Length,No. ENS lincrna Exons which overlap RefSeq exons,Max ORF length(ENS),Max No. of amino acids(ENS),Max Transcript Index for ORF(ENS),GC % of trans row having Max ORF(ENS),Max ORF Sequence(ENS)
0,ENSG00000236648,gene-LOC107984028,1,1,4,Yes,1,17270,3,90,30,153,51.97,ATGGCTGCAAATGGCTCCTCCGTGCAGGTGAGGACATGGAAGCACA...
1,ENSG00000230704,gene-LOC101928628,1,1,1,Yes,1,1720,2,36,12,818,40.92,ATGTATTCCTCTCTCTCTCTCACCTATTCCCATGTTTAA


In [33]:
from tqdm import tqdm
l_id = []
r_id = []
for i in tqdm(range(len(df_overlap['ENS lincRNA gene ID']))):
    l_id.append(df_overlap['ENS lincRNA gene ID'][i])
    r_id.append(df_overlap['Ref lncRNA gene ID'][i])

100%|██████████| 4557/4557 [00:00<00:00, 19075.18it/s]


In [34]:
set_lincrna_id = set(l_id)
set_ref_id = set(r_id)

In [35]:
df_ref_transcript_data = pd.read_csv('/feature_selection/ORF/RefSeq/all_lncrna_transcript_data_orf_info.csv', index_col=0)

In [36]:
drop_index = []
drop_no = 0
for i in tqdm(range(len(df_ref_transcript_data))):
    if df_ref_transcript_data['Len of longest ORF'][i] == 'No start codon found' or df_ref_transcript_data['Len of longest ORF'][i] == 'Nan':
        drop_index.append(i)
        drop_no = drop_no + 1

100%|██████████| 25715/25715 [00:01<00:00, 18699.81it/s]


In [37]:
df_ref_transcript_data.drop(drop_index, axis=0,inplace=True)
df_ref_transcript_data.reset_index(drop=True, inplace=True)

In [38]:
len(df_ref_transcript_data)

25689

In [39]:
df_ref_transcript_data["Len of longest ORF"] = pd.to_numeric(df_ref_transcript_data["Len of longest ORF"])

In [40]:
df_ref_transcript_data["GC %"] = pd.to_numeric(df_ref_transcript_data["GC %"])

In [41]:
df_ref_transcript_data.head(3)

,SNO,Index,RefSeq Gene ID,RefSeq Transcript ID,Dbxref,Parent,gbkey,Type,Seq_id,Strand,...,Match ?,Sequences,GC %,Len of longest ORF,No. of Amino Acids,Longest ORF Seq,ORF Start Index for longest Seq,Protein Seq encoded,Stop codon present ?,ORF multiple of 3 ?
0,1,27,gene-MIR1302-2HG,rna-XR_001737835.1,"GeneID:107985730,Genbank:XR_001737835.1,HGNC:H...",gene-MIR1302-2HG,ncRNA,lnc_RNA,1,1,...,YES,TCGTATGAGTCCAGTTTTGTGAAGTGCCTAGAATAGTCAAAATTCA...,51.30,15,5,ATGAGTCCAGTTTTGTGA,4,MSPVL,YES,YES
1,1508,251564,gene-LOC107985451,rna-XR_001737807.1,"GeneID:107985451,Genbank:XR_001737807.1",gene-LOC107985451,ncRNA,lnc_RNA,1,-1,...,YES,GCCATTAAGGGATGAGCGTAAGCAGTGACTGACAGGCAGGCTCTGC...,51.74,18,6,ATGAGCGTAAGCAGTGACTGA,11,MSVSSD,YES,YES
2,1507,251556,gene-LOC100505795,rna-NR_038072.1,"GeneID:100505795,Genbank:NR_038072.1",gene-LOC100505795,ncRNA,lnc_RNA,1,-1,...,YES,CTTATTAAAAATAGAATATTGCCTACATCACATAAAGGGCAAACTC...,44.27,123,41,ATGGGCACACTTGGTGATGTGCAACTCAGTGGGGCCTACAGGGTCC...,118,MGTLGDVQLSGAYRVPADIDITKEFEPGLRNRVSGCFKKAV,YES,YES


In [42]:
cleaned_max_orf = []
cleaned_amino_acid = []
cleaned_max_index = []
cleaned_gc_percentage = []
cleaned_max_orf_seq = []

In [43]:
def get_gene_orf():
    for i in tqdm(range(len(df_overlap))):
        a = df_ref_transcript_data.index[df_ref_transcript_data['RefSeq Gene ID'] == df_overlap['Ref lncRNA gene ID'][i]]
        max_orf = 0
        max_gc = 0
        index_max_all = 0
        for i in range(len(a)):
            if df_ref_transcript_data.iloc[a[i]]['Len of longest ORF'] > max_orf:
                max_orf = df_ref_transcript_data.iloc[a[i]]['Len of longest ORF']
                max_gc = df_ref_transcript_data.iloc[a[i]]['GC %']
                index_max_all = a[i]
            elif df_ref_transcript_data.iloc[a[i]]['Len of longest ORF'] == max_orf:
                if df_ref_transcript_data.iloc[a[i]]['GC %'] > max_gc:
                    max_gc = df_ref_transcript_data.iloc[a[i]]['GC %']
                    index_max_all = a[i]
        
        cleaned_max_index.append(index_max_all)
        cleaned_max_orf.append(max_orf)
        cleaned_amino_acid.append(df_ref_transcript_data.iloc[index_max_all]['No. of Amino Acids'])
        cleaned_gc_percentage.append(max_gc)
        cleaned_max_orf_seq.append(df_ref_transcript_data.iloc[index_max_all]['Longest ORF Seq'])

In [44]:
%%time
get_gene_orf()

100%|██████████| 4557/4557 [01:02<00:00, 73.19it/s] 

CPU times: user 47 s, sys: 1.71 s, total: 48.7 s
Wall time: 1min 2s


In [45]:
print(cleaned_max_orf[0:2])
print(cleaned_gc_percentage[0:2])
print(cleaned_amino_acid[0:2])

[231, 9]
[49.57, 39.04]
['77', '3']


In [46]:
df_overlap['Max ORF length(REF)'] = cleaned_max_orf
df_overlap['Max No. of amino acids(REF)'] = cleaned_amino_acid
df_overlap['Max Transcript Index for ORF(REF)'] = cleaned_max_index
df_overlap['GC % of trans row having Max ORF(REF)'] = cleaned_gc_percentage
df_overlap['Max ORF Sequence(REF)'] = cleaned_max_orf_seq

In [47]:
df_overlap.to_csv('/feature_selection/ORF/Ens-Ref/ens_ref_ORF&overlap_cleaned(ens&ref).csv')